In [7]:
# Exercise 2.1
import tiktoken

# Initialize the tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

# Tokenize a made-up string
token_ids = tokenizer.encode("Akwirw ier")
print(token_ids)

# Examine each token individually
for token_id in token_ids:
    print(f"{token_id} -> {tokenizer.decode([token_id])}")

# Verify individual tokens
print(tokenizer.encode("Ak"))
print(tokenizer.encode("w"))
print(tokenizer.encode("ir"))
print(tokenizer.encode("w"))
print(tokenizer.encode(" "))
print(tokenizer.encode("ier"))

# Verify decoding works correctly
print(tokenizer.decode([33901, 86, 343, 86, 220, 959]))

[33901, 86, 343, 86, 220, 959]
33901 -> Ak
86 -> w
343 -> ir
86 -> w
220 ->  
959 -> ier
[33901]
[86]
[343]
[86]
[220]
[959]
Akwirw ier


In [5]:
# Exercise 2.2
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDataset(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        all_tokens = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        # Create sliding windows of tokens
        for i in range(0, len(all_tokens) - max_length, stride):
            # Input is current window
            x = all_tokens[i:i + max_length]
            # Target is shifted by 1 (next token prediction)
            y = all_tokens[i+1:i + max_length + 1]

            self.input_ids.append(torch.tensor(x))
            self.target_ids.append(torch.tensor(y))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

def create_dataloader(text_file, batch_size=4, max_length=256, stride=128):
    # Read text file
    with open(text_file, "r", encoding="utf-8") as f:
        text = f.read()

    # Initialize tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDataset(text, tokenizer, max_length, stride)

    # Create and return dataloader
    return DataLoader(dataset, batch_size=batch_size)

# Test the dataloader with small context windows
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Test with small context length
dataloader_small = create_dataloader("the-verdict.txt", batch_size=4, max_length=2, stride=2)
for batch in dataloader_small:
    inputs, targets = batch
    print("Small context inputs:")
    print(inputs)
    break

# Test with larger context length
dataloader_large = create_dataloader("the-verdict.txt", batch_size=4, max_length=8, stride=2)
for batch in dataloader_large:
    inputs, targets = batch
    print("Larger context inputs:")
    print(inputs)
    break

# Setup for embedding layers
vocab_size = 50257  # GPT-2 vocabulary size
embedding_dim = 256
context_length = 4

# Create embedding layers
token_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
position_embeddings = torch.nn.Embedding(context_length, embedding_dim)

Small context inputs:
tensor([[  40,  367],
        [2885, 1464],
        [1807, 3619],
        [ 402,  271]])
Larger context inputs:
tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271],
        [ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138],
        [ 1807,  3619,   402,   271, 10899,  2138,   257,  7026],
        [  402,   271, 10899,  2138,   257,  7026, 15632,   438]])
